## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-15-05-22-38 +0000,nypost,"Nevada boy, 11, shot dead by gun-toting lunati...",https://nypost.com/2025/11/15/us-news/nevada-b...
1,2025-11-15-05-14-22 +0000,nypost,Mega Millions ticket holder wins $980M jackpot,https://nypost.com/2025/11/15/us-news/mega-mil...
2,2025-11-15-04-41-00 +0000,bbc,Trump says he will take legal action against B...,https://www.bbc.com/news/articles/c891jp9j79do...
3,2025-11-15-04-39-50 +0000,nyt,Modi’s Coalition Sweeps Indian State Election ...,https://www.nytimes.com/2025/11/14/world/asia/...
4,2025-11-15-04-23-27 +0000,nyt,"Trump Cuts Ties With Marjorie Taylor Greene, C...",https://www.nytimes.com/2025/11/14/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2386/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,trump,50
81,epstein,17
349,new,12
20,will,11
19,he,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
16,2025-11-15-03-01-00 +0000,wsj,President Trump said he is withdrawing his sup...,https://www.wsj.com/politics/policy/trump-with...,133
43,2025-11-15-01-02-00 +0000,wsj,Trump said he would ask the Justice Department...,https://www.wsj.com/politics/policy/trump-call...,131
132,2025-11-14-21-07-00 +0000,wsj,Trump said he is withdrawing the nomination of...,https://www.wsj.com/politics/policy/trump-with...,103
207,2025-11-14-16-16-56 +0000,nypost,Goldman Sachs stands by top lawyer who bashed ...,https://nypost.com/2025/11/14/business/goldman...,100
9,2025-11-15-03-59-10 +0000,nypost,Trump withdraws endorsement of Rep. Marjorie T...,https://nypost.com/2025/11/14/us-news/trump-wi...,99


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
16,133,2025-11-15-03-01-00 +0000,wsj,President Trump said he is withdrawing his sup...,https://www.wsj.com/politics/policy/trump-with...
132,41,2025-11-14-21-07-00 +0000,wsj,Trump said he is withdrawing the nomination of...,https://www.wsj.com/politics/policy/trump-with...
119,41,2025-11-14-21-47-00 +0000,wsj,The reopened federal government will on Thursd...,https://www.wsj.com/economy/jobs/september-job...
143,40,2025-11-14-20-10-04 +0000,nyt,Israeli Settler Attack During West Bank Olive ...,https://www.nytimes.com/2025/11/14/world/middl...
95,39,2025-11-14-22-26-25 +0000,nypost,Jack Schlossberg shamelessly invokes Kennedy n...,https://nypost.com/2025/11/14/us-news/jack-sch...
218,38,2025-11-14-15-36-00 +0000,wsj,Who Is John Furner? Meet the Walmart Insider N...,https://www.wsj.com/business/retail/john-furne...
134,35,2025-11-14-21-03-39 +0000,nypost,Purdue Pharma’s $7B opioid bankruptcy deal to ...,https://nypost.com/2025/11/14/business/judge-a...
77,35,2025-11-14-22-56-00 +0000,wsj,"Trump moved to lower duties on beef, coffee an...",https://www.wsj.com/economy/trade/trump-implem...
305,35,2025-11-14-09-00-00 +0000,cbc,5-year-old Masa has been missing for more than...,https://www.cbc.ca/news/world/gaza-rubble-bodi...
214,32,2025-11-14-16-00-00 +0000,wsj,"Our compulsive purchasing of cheap, foreign-ma...",https://www.wsj.com/economy/consumers/will-tru...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
